![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [6]:
import json
import numpy 
import matplotlib.pyplot as plt
import pandas as pd
import itertools

qb = QuantBook()
prices = json.loads(qb.ObjectStore.Read("MyObject"))
qb.error={}
corr={}
count=1

#Convert our dictionary which is saved in ObjectStore into a dataframe 
#We only drop contracts which have no data at all hence the 'all'
var =pd.DataFrame.from_dict(prices,orient="index").dropna(how='all')

for row in var.itertuples():
    templist=[]
    vol=0
    #i represents each minute
    for i in range(1,len(row)):
        vol=vol+row[i][3]
        #calculate total volume
        spread=float(row[i][1])+float(row[i][2])/2
        #mid-spread price
        if spread!=0:
            templist.append(abs( (spread-float(row[i][0])/spread)))
            lastprice=(spread-float(row[i][0])/spread)
            #append the error between last price and theoretical price for each contract in a time series
        qb.error[row[0]]=(templist)
        #save the error each timestep
    count+=1   

    

In [19]:
val=[]    
for x in qb.error.keys():
    val.append(x)
    #retrieve contract symbols
newcount=1
for combination in itertools.combinations(val, 2):
    #take all possible combinations of 2, ignore all except straddles
    if ( combination[0][15:20])!= ( combination[1][15:20]) or  ( combination[0][12])==( combination[1][12]):
        continue
    #calculates correlation between put and call, plots
    corr[combination]=numpy.corrcoef(qb.error[combination[0]],qb.error[combination[1]])
    plt.figure(newcount)
    plt.plot(qb.error[combination[0]])
    plt.plot(qb.error[combination[1]])
    plt.title(combination)
    plt.xlabel(str('Corr:')+str(corr[combination][0][1]))
    newcount+=1
